In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
# 1. Import the data from medical_examination.csv and assign it to the df variable.
df = pd.read_csv('medical_examination.csv')

# 2. Add an overweight column to the data.
#    BMI = weight (kg) / (height (m))^2 ; overweight if BMI > 25.
df['overweight'] = ((df['weight'] / ((df['height'] / 100) ** 2)) > 25).astype(int)

# 3. Normalize data: make 0 always good and 1 always bad for cholesterol and gluc.
df['cholesterol'] = (df['cholesterol'] > 1).astype(int)
df['gluc'] = (df['gluc'] > 1).astype(int)


def draw_cat_plot():
    """
    4–8. Draw the Categorical Plot showing counts of good (0) and bad (1)
    outcomes for selected features, split by cardio in separate panels.
    Returns the matplotlib Figure object.
    """
    # 5. Create df_cat via pd.melt for the chosen categorical variables.
    df_cat = pd.melt(
        df,
        id_vars=['cardio'],
        value_vars=['cholesterol', 'gluc', 'smoke', 'alco', 'active', 'overweight']
    )

    # 6. Group and reformat to get counts.
    df_cat = (
        df_cat
        .groupby(['cardio', 'variable', 'value'])
        .size()
        .reset_index(name='total')  # rename count column for catplot
    )

    # 7. Convert the data into long format and create the catplot.
    g = sns.catplot(
        data=df_cat,
        x='variable', y='total', hue='value', col='cardio',
        kind='bar', height=5, aspect=1
    )
    g.set_axis_labels('variable', 'total')
    g._legend.set_title('value')

    # 8. Get the figure for output and store it in fig.
    fig = g.fig

    # Do not modify the next two lines.
    fig.savefig('catplot.png')
    return fig


def draw_heat_map():
    """
    9–15. Draw a heatmap of the correlation matrix after cleaning the data
    for physiological plausibility and outliers. Returns the Figure.
    """
    # 10. Clean the data in df_heat by filtering out incorrect data.
    df_heat = df[(df['ap_lo'] <= df['ap_hi']) &
                 (df['height'] >= df['height'].quantile(0.025)) &
                 (df['height'] <= df['height'].quantile(0.975)) &
                 (df['weight'] >= df['weight'].quantile(0.025)) &
                 (df['weight'] <= df['weight'].quantile(0.975))]

    # 11. Calculate the correlation matrix.
    corr = df_heat.corr(numeric_only=True)

    # 12. Generate a mask for the upper triangle.
    mask = np.triu(np.ones_like(corr, dtype=bool))

    # 13. Set up the matplotlib figure.
    fig, ax = plt.subplots(figsize=(12, 10))

    # 14. Plot the correlation matrix using seaborn heatmap.
    sns.heatmap(
        corr,
        mask=mask,
        annot=True,
        fmt='.1f',
        square=True,
        center=0.0,
        linewidths=.5,
        cbar_kws={'shrink': .5}
    )

    # 15. Do not modify the next two lines.
    fig.savefig('heatmap.png')
    return fig


if __name__ == '__main__':
    # Optional: run both plots when executing the script directly.
    draw_cat_plot()
    draw_heat_map()
